In [35]:
# session.write_pandas(df,table_name="HR_CHURN",auto_create_table=True).show()
# !pip install fosforml

In [5]:
from fosforml.model_manager.snowflakesession import get_session,get_connection_params
session = get_session()
connection_params = get_connection_params()

In [6]:
import pandas as pd
df = pd.read_csv("HR_CHURN.csv",index_col='Unnamed: 0')

from snowflake.snowpark import Session
		
# session = Session.builder.configs(CONNECTION_PARAMETERS).create()

In [7]:
from snowflake.snowpark.session import Session,col
import snowflake.snowpark.functions
from snowflake.ml.modeling.metrics import confusion_matrix, accuracy_score, f1_score, recall_score,precision_score
from snowflake.ml.modeling.preprocessing import LabelEncoder, StandardScaler
from snowflake.ml.modeling.impute import SimpleImputer
from snowflake.ml.modeling.ensemble import GradientBoostingClassifier,RandomForestClassifier,RandomForestRegressor
from snowflake.ml.modeling.xgboost import XGBClassifier
from snowflake.ml.modeling.pipeline import Pipeline
import snowflake
def apply_label_encoding(input_data_frame):
    for i in input_data_frame.dtypes:
        if i[1].find('string') >= 0:
            label_encoder = LabelEncoder(input_cols=i[0],output_cols=i[0],drop_input_cols=True)
            input_data_frame = label_encoder.fit(input_data_frame).transform(input_data_frame)
    return input_data_frame

In [8]:
snow_df = session.create_dataframe(df)
snow_df = snow_df.drop(['Unnamed: 0'])
input_data_frame, test_df = snow_df.randomSplit([0.75, 0.25])
label_encoder_column = []     
input_data_frame = apply_label_encoding(input_data_frame)
test_df = apply_label_encoding(test_df)
feature_cols = input_data_frame.columns
target_col = "SALARY"
feature_cols.remove(target_col)
OUTPUT_COLS = [target_col + '_PREDICTION']

/opt/conda/lib/python3.9/site-packages/snowflake/snowpark/session.py:2484: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.base.Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  success, _, _, ci_output = write_pandas(


In [9]:
pipeline = GradientBoostingClassifier(input_cols=feature_cols, label_cols=target_col)
pipeline.fit(input_data_frame)
pred_snow_df = pipeline.predict(test_df)

Got error object of type 'NoneType' has no len() when trying to read default values from function: <function SnowparkModelTrainer._build_fit_wrapper_sproc.<locals>.fit_wrapper_function at 0x7f98876cc4c0>. Proceeding without creating optional arguments
The version of package 'snowflake-snowpark-python' in the local environment is 1.21.1, which does not fit the criteria for the requirement 'snowflake-snowpark-python'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn==1.3.0'. Your UDF might not work when the package version is different between the server and your local environment.
/opt/conda/lib/python3.9/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator DummyClassifier from version 1.3.0 when using version 1.3.2. This might lead to breaking code or 

In [10]:
from fosforml import register_model

register_model(
  model_obj=pipeline,
  session=session,
  name="TestModelForDrifts",
  snowflake_df=pred_snow_df,
  dataset_name="HR_CHURN",
  dataset_source="Dataset",
  source="NOTEBOOK",
  description="This is a Snowflake model",
  flavour="snowflake",
  model_type="classification",
  conda_dependencies=["scikit-learn==1.3.2"]
)

Got error object of type 'NoneType' has no len() when trying to read default values from function: <class 'snowflake.ml.modeling.metrics.classification._register_confusion_matrix_computer.<locals>.ConfusionMatrixComputer'>. Proceeding without creating optional arguments
Got error object of type 'NoneType' has no len() when trying to read default values from function: <class 'snowflake.ml.modeling.metrics.metrics_utils.register_accumulator_udtf.<locals>.Accumulator'>. Proceeding without creating optional arguments


Calculating build time metrics

Progress: ██████████████                                                         20.0%
Calculating build time metrics

Progress: ████████████████████████████                                           40.0%


Got error object of type 'NoneType' has no len() when trying to read default values from function: <class 'snowflake.ml.modeling.metrics.classification._register_multilabel_confusion_matrix_computer.<locals>.MultilabelConfusionMatrixComputer'>. Proceeding without creating optional arguments
The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn<1.4'. Your UDF might not work when the package version is different between the server and your local environment.
DataFrame.flatten() is deprecated since 0.7.0. Use `DataFrame.join_table_function()` instead.
Got error object of type 'NoneType' has no len() when trying to read default values from function: <class 'snowflake.ml.modeling.metrics.classification._register_multilabel_confusion_matrix_computer.<locals>.MultilabelConfusionMatrixComputer'>. Proceeding without creating optional arguments
The version of package 'scikit-learn' in the local environment is 1.

Calculating build time metrics

Progress: ██████████████████████████████████████████                             60.0%
Calculating build time metrics

Progress: ████████████████████████████████████████████████████████               80.0%
Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


/opt/conda/lib/python3.9/contextlib.py:119: UserWarning: `relax_version` is not set and therefore defaulted to True. Dependency version constraints relaxed from ==x.y.z to >=x.y, <(x+1). To use specific dependency versions for compatibility, reproducibility, etc., set `options={'relax_version': False}` when logging the model.
  return next(self.gen)
/opt/conda/lib/python3.9/site-packages/snowflake/ml/model/_packager/model_packager.py:92: UserWarning: Inferring model signature from sample input or providing model signature for Snowpark ML Modeling model is not required. Model signature will automatically be inferred during fitting. 
  handler.save_model(


"Failed to save build time metrics for model 'MODEL_8D3FB8A6_A886_4236_9B35_73EB1303C4BF_FDC_TESTMODELFORDRIFTS'. No module named 'model_manager'"